## Single Callback for Interactivity

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
app = dash.Dash()

In [ ]:
app.layout = html.Div([
    
    dcc.Input(id='in-id1',value='Init text',type='text'),
    html.Div([html.P()],id='div-id1',style={'border':'2px blue solid'})
    
    
])

@app.callback(Output(component_id='div-id1',component_property='children'),
              [Input(component_id='in-id1',component_property='value')])
def update_output_div(in_val):
    return('{} me?'.format(in_val) + ' Well {} you too!'.format(in_val))

In [ ]:
app.run_server()

### Callbacks for Graphs

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd

In [ ]:
df = pd.read_csv('../Plotly-Dashboards-with-Dash/Data/gapminderDataFiveYear.csv')

In [ ]:
countries = list(map(lambda x: {'label':str(x),'value':x},df['country'].unique()))

In [ ]:
app = dash.Dash()

In [ ]:
app.layout = html.Div([
    dcc.Graph(id='graph'),
    dcc.Dropdown(id='con-picker',options = countries,value=['United States'],multi=True)
])

In [ ]:
@app.callback(Output('graph','figure'),
              [Input('con-picker','value')])
def update_figure(values):
    sub_df = df[df['country'].isin(values)]
    
    traces = []
    for each in sub_df['country'].unique():
        temp = sub_df[sub_df['country']==each]

        traces.append(go.Scattergl(
                            x=temp['year'],
                            y=temp['lifeExp'],
                            mode='lines',
                            opacity=0.7,
                            name=each
                        ))
    
    return({'data':traces,
            'layout':go.Layout(
                               xaxis={'title':'Year'},
                               yaxis={'title':'Life Expentancy'})})

In [ ]:
app.run_server()

### Multiple Inputs

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd

In [ ]:
df = pd.read_csv('../Plotly-Dashboards-with-Dash/Data/mpg.csv')

In [ ]:
app = dash.Dash()

In [ ]:
features = df.columns

In [ ]:
app.layout = html.Div([
    
    html.Div(['x-axis',
        dcc.Dropdown(id='xaxis',
                     options=list(map(lambda x: {'label':str(x),'value':x},features)),
                     value='mpg')
    ],
        style={'width':'40%','display':'inline-block'}),
    
    html.Div(['y-axis',
        dcc.Dropdown(id='yaxis',
                     options=list(map(lambda x: {'label':str(x),'value':x},features)),
                     value='weight')
    ],
        style={'width':'40%','display':'inline-block'}),
    
    html.Div([
        dcc.Graph(id='feature_graph')
    ])
    
],
        style={'padding':10})

In [ ]:
@app.callback(Output('feature_graph','figure'),
              [Input('xaxis','value'),
               Input('yaxis','value')])
def update_graph(xaxis_name,yaxis_name):
    
    figure_dict = {'data':
                   [
                       go.Scattergl(x=df[xaxis_name],
                                    y=df[yaxis_name],
                                    mode='markers',
                                    text=df['name'],
                                    opacity=0.7,
                                    marker={'size':15}
                                    )
                   ]
                   ,
                   'layout':go.Layout(xaxis={'title':xaxis_name},yaxis={'title':yaxis_name},
                                      hovermode='closest')}
    return(figure_dict)

In [ ]:
app.run_server()

### Multiple Outputs

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd
import base64

In [ ]:
df = pd.read_csv('../Plotly-Dashboards-with-Dash/Data/wheels.csv')

app = dash.Dash()

# prep image
def encode_image(image_file):
    encoded = base64.b64encode(open(image_file,'rb').read())
    return('data:image/png;base64,{}'.format(encoded.decode()))

In [ ]:
app.layout = html.Div([
    dcc.RadioItems(
        id='wheels',
        options=[{'label': i, 'value': i} for i in df['wheels'].unique()],
        value=1
    ),
    html.Div(id='wheels-output'),

    html.Hr(),  # add a horizontal rule
    dcc.RadioItems(
        id='colors',
        options=[{'label': i, 'value': i} for i in df['color'].unique()],
        value='blue'
    ),
    html.Div(id='colors-output'),
    html.Img(id='display-image', src='children', height=300)
], style={'fontFamily':'helvetica', 'fontSize':18})

In [ ]:
@app.callback(
    Output('wheels-output', 'children'),
    [Input('wheels', 'value')])
def callback_a(wheels_value):
    return 'You\'ve selected "{}"'.format(wheels_value)

@app.callback(
    Output('colors-output', 'children'),
    [Input('colors', 'value')])
def callback_b(colors_value):
    return 'You\'ve selected "{}"'.format(colors_value)

@app.callback(
    Output('display-image', 'src'),
    [Input('wheels', 'value'),
     Input('colors', 'value')])
def callback_image(wheel, color):
    path = '/home/mandeep/workspace/learning/plotly/udemy/Plotly-Dashboards-with-Dash/Data/Images/'
    temp = encode_image(path+df[(df['wheels']==wheel) & (df['color']==color)]['image'].values[0])
    return(temp)

In [ ]:
app.run_server()